In [24]:

import yfinance as yf
import pandas as pd
import pandas_market_calendars as mcal
import numpy as np
import clickhouse_connect as Client
import os

In [3]:
def download_single_ticker_data(ticker, output_file=None):
    """
    下载单个Ticker（股票或ETF）的历史数据。
    
    :param ticker: 股票或ETF代码（如 "SPY"）
    :param start_date: 数据开始日期（如 "2022-01-01"）
    :param end_date: 数据结束日期（如 "2023-01-01"）
    :param output_file: 可选，保存文件路径（如 "ticker_data.csv"）
    :return: 包含数据的Pandas DataFrame
    """
    print(f"正在下载 {ticker} 的数据...")
    
    # 使用 yfinance 下载数据
    data = yf.download(ticker)
    
    # 检查数据是否下载成功
    if data.empty:
        print(f"无法下载 {ticker} 的数据，请检查代码或日期范围。")
        return None
    
    # 如果需要保存到文件
    if output_file:
        data.to_csv(output_file)
        print(f"数据已保存到文件：{output_file}")
    
    return data


In [20]:
def init_db_df(symbol, start, end, freq='1D'):
    
    # Create a calendar
    nasdaq = mcal.get_calendar('NASDAQ')
    trading_hours = pd.DataFrame()
    trading_hours['start'] = nasdaq.schedule(start_date=start, end_date=end)['market_open'].dt.tz_convert('America/New_York')
    trading_hours['end'] = nasdaq.schedule(start_date=start, end_date=end)['market_close'].dt.tz_convert('America/New_York')
    # trading_hours['duration'] = (trading_hours['end'] - trading_hours['start']).dt.total_seconds() / 60
    ETH = []
    for _, row in trading_hours.iterrows():
        ETH.append(pd.date_range(start=row['start'], end=row['end'], freq='T', inclusive="left").tz_localize(None))
    ETH = pd.DatetimeIndex(np.concatenate(ETH))
    # trading_hours = trading_hour[trading_hour['duration'] == 390]
    # minute_ranges = []
    # for _, row in trading_hour.iterrows():
    #     # Generate minute-frequency datetime range
    #     time_range = pd.date_range(start=row['start'], end=row['end'], freq='T', inclusive="left").tz_localize(None)
    #     minute_ranges.append(time_range)
    # minute_ranges = pd.DatetimeIndex(np.concatenate(minute_ranges))

    columns = 'symbol, open_time_local, open, high, low, close, volume, average'.split(', ')
    sql = f"SELECT symbol, open_time_local, open, high, low, close, volume, average from us_stocks.pg_klines_1m WHERE symbol = '{symbol}' AND open_time_local >= '{start}' AND open_time_local < '{end}';"
    client = Client.get_client(host='192.168.1.19', username='root', password='flab777')
    result = client.query(sql)
    df = pd.DataFrame(result.result_rows, columns=columns).sort_values('open_time_local')
    df['open_time'] = pd.to_datetime(df['open_time_local'], format='%Y%m%d %H:%M:%S')
    df['open'] = df['open'].astype(float)
    df['high'] = df['high'].astype(float)
    df['low'] = df['low'].astype(float)
    df['close'] = df['close'].astype(float)
    df['volume'] = df['volume'].astype(float)
    df.drop_duplicates(inplace=True)
    df.set_index('open_time', inplace=True, drop=False)
    df.dropna(inplace=True)
    df = df[df.index.isin(ETH)]

    df = df.resample(freq).agg({
        'symbol': 'first',
        'open_time': 'first',
        'open': 'first',
        'high': 'max',
        'low': 'min',
        'close': 'last',
        'volume': 'sum',
    })
    df.dropna(inplace=True)

    # Group by date
    # start_time = pd.Timestamp("09:30:00").time()
    # end_time = pd.Timestamp("15:59:00").time()

    df['date'] = df.index.date
    # df = df[df.date.isin(trading_hours.index.date)]
    df.set_index('date', inplace=True, drop=False)
    # df = df.reindex(minute_ranges)

    return df



In [27]:
ETFs=pd.read_csv("../data/ETF1s.csv")
for etf in ETFs['Symbol']:
    if os.path.exists(f"../data/daily_data/{etf}_daily.csv"):
        continue
    print(etf)
    etfKline=init_db_df(etf,"2010-01-01","2025-02-01")
    etfKline.to_csv(f"../data/daily_data/{etf}_daily.csv")

SPTL


/tmp/ipykernel_2473466/2725158429.py:11: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  ETH.append(pd.date_range(start=row['start'], end=row['end'], freq='T', inclusive="left").tz_localize(None))


SCHR


/tmp/ipykernel_2473466/2725158429.py:11: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  ETH.append(pd.date_range(start=row['start'], end=row['end'], freq='T', inclusive="left").tz_localize(None))


SOXL


...

/tmp/ipykernel_2473466/2725158429.py:11: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  ETH.append(pd.date_range(start=row['start'], end=row['end'], freq='T', inclusive="left").tz_localize(None))


LIAX


/tmp/ipykernel_2473466/2725158429.py:11: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  ETH.append(pd.date_range(start=row['start'], end=row['end'], freq='T', inclusive="left").tz_localize(None))


LIAO


/tmp/ipykernel_2473466/2725158429.py:11: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  ETH.append(pd.date_range(start=row['start'], end=row['end'], freq='T', inclusive="left").tz_localize(None))


In [6]:
ETFs=pd.read_csv('../data/ETFs.csv')

download_single_ticker_data('SPY', 'SPY.csv')
# for etf in ETFs['Symbol']:
    # download_single_ticker_data(etf,  f"{etf}.csv")

正在下载 SPY 的数据...


Failed to get ticker 'SPY' reason: Expecting value: line 1 column 1 (char 0)
[*********************100%***********************]  1 of 1 completed

1 Failed download:
['SPY']: YFTzMissingError('$%ticker%: possibly delisted; no timezone found')


无法下载 SPY 的数据，请检查代码或日期范围。
